#Artificial Neural Networks And Deep Learning
#Homework 1

#Students: 

*   Julián Jiménez
*   Samuel Polo



In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
import numpy as np
import os

In [3]:
# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()
print(cwd)

/content


In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Classification_Dataset.zip to Classification_Dataset.zip
User uploaded file "Classification_Dataset.zip" with length 94923108 bytes


In [5]:
!unzip Classification_Dataset.zip

Archive:  Classification_Dataset.zip
  inflating: Classification_Dataset/test/IMG_0.jpg  
  inflating: Classification_Dataset/test/IMG_1001.jpg  
  inflating: Classification_Dataset/test/IMG_1007.jpg  
  inflating: Classification_Dataset/test/IMG_1011.jpg  
  inflating: Classification_Dataset/test/IMG_1023.jpg  
  inflating: Classification_Dataset/test/IMG_1031.jpg  
  inflating: Classification_Dataset/test/IMG_1035.jpg  
  inflating: Classification_Dataset/test/IMG_1037.jpg  
  inflating: Classification_Dataset/test/IMG_104.jpg  
  inflating: Classification_Dataset/test/IMG_1042.jpg  
  inflating: Classification_Dataset/test/IMG_1046.jpg  
  inflating: Classification_Dataset/test/IMG_105.jpg  
  inflating: Classification_Dataset/test/IMG_1050.jpg  
  inflating: Classification_Dataset/test/IMG_1052.jpg  
  inflating: Classification_Dataset/test/IMG_1058.jpg  
  inflating: Classification_Dataset/test/IMG_1059.jpg  
  inflating: Classification_Dataset/test/IMG_106.jpg  
  inflating: Clas

In [0]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.2)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255
                                        ,validation_split=0.2)

valid_data_gen = ImageDataGenerator(rescale=1./255)

In [0]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'Classification_Dataset')

#Batch Size
bs = 64
# img shape
img_h = 256
img_w = 256

num_classes = 20

classes = ['owl',              # 0
           'galaxy',           # 1
           'lightning',        # 2
           'wine-bottle',      # 3
           't-shirt',          # 4
           'waterfall',        # 5
           'sword',            # 6
           'school-bus',       # 7
           'calculator',       # 8
           'sheet-music',      # 9
           'airplanes',        # 10
           'lightbulb',        # 11
           'skyscraper',       # 12
           'mountain-bike',    # 13
           'fireworks',        # 14
           'computer-monitor', # 15
           'bear',             # 16
           'grand-piano',      # 17
           'kangaroo',         # 18
           'laptop']           # 19

In [30]:
# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset='training',
                                               seed=SEED)

Found 1247 images belonging to 20 classes.


In [31]:
# Validation
# Here we can use the previous code for separating the validation folder.
##-----------------------------------------------------------------------
valid_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               subset='validation',
                                               seed=SEED)

Found 307 images belonging to 20 classes.


In [32]:
#Create Dataset objects
#Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()
print(len(train_gen))

20


In [0]:
# Validation 
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
# Repeat
valid_dataset = valid_dataset.repeat()

#**Transfer Learning** 
##**Our choice: VGG16**

In [0]:
# Load VGG16 Model

vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [35]:
vgg.summary()
vgg.layers

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [36]:
# Create Model
# ------------

finetuning = False #We didn't use Finetuning because we were provided little training data 
#and the pre-trained VGG16 Model is expected to match this Classification problem.

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 4096)              134221824 
_________________________________________________________________
dense_8 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_9 (Dense)              (None, 20)                81940     
Total params: 165,799,764
Trainable params: 151,085,076
Non-trainable params: 14,714,688
_________________________________________________________________


[<tf.Variable 'block1_conv1_2/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
 array([[[[ 4.29470569e-01,  1.17273867e-01,  3.40129584e-02, ...,
           -1.32241577e-01, -5.33475243e-02,  7.57738389e-03],
          [ 5.50379455e-01,  2.08774377e-02,  9.88311544e-02, ...,
           -8.48205537e-02, -5.11389151e-02,  3.74943428e-02],
          [ 4.80015397e-01, -1.72696680e-01,  3.75577137e-02, ...,
           -1.27135560e-01, -5.02991639e-02,  3.48965675e-02]],
 
         [[ 3.73466998e-01,  1.62062630e-01,  1.70863140e-03, ...,
           -1.48207128e-01, -2.35300660e-01, -6.30356818e-02],
          [ 4.40074533e-01,  4.73412387e-02,  5.13819456e-02, ...,
           -9.88498852e-02, -2.96195745e-01, -7.04357103e-02],
          [ 4.08547401e-01, -1.70375049e-01, -4.96297423e-03, ...,
           -1.22360572e-01, -2.76450396e-01, -3.90796512e-02]],
 
         [[-6.13601133e-02,  1.35693997e-01, -1.15694344e-01, ...,
           -1.40158370e-01, -3.77666801e-01, -3.00509870e-01],
  

##**Prepare the model for Training** 

In [0]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

##**Training with Callbacks**

In [38]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'transfer_learning_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Train for 20 steps, validate for 5 steps
Epoch 1/100
20/20 [==============================] - 49s 2s/step - loss: 15.1687 - accuracy: 0.1026 - val_loss: 2.6041 - val_accuracy: 0.1889
Epoch 2/100
20/20 [==============================] - 48s 2s/step - loss: 1.7163 - accuracy: 0.4507 - val_loss: 1.3074 - val_accuracy: 0.5505
Epoch 3/100
20/20 [==============================] - 48s 2s/step - loss: 0.9223 - accuracy: 0.7081 - val_loss: 0.9150 - val_accuracy: 0.6971
Epoch 4/100
20/20 [==============================] - 48s 2s/step - loss: 0.6981 - accuracy: 0.7739 - val_loss: 0.9782 - val_accuracy: 0.6678
Epoch 5/100
20/20 [==============================] - 48s 2s/step - loss: 0.5178 - accuracy: 0.8284 - val_loss: 0.9283 - val_accuracy: 0.7199
Epoch 6/100
20/20 [==============================] - 48s 2s/step - loss: 0.4520 - accuracy: 0.8581 - val_loss: 0.9590 - val_accuracy: 0.7101
Epoch 7/100
20/20 [==============================] - 48s 2s/step - loss: 0.3945 - accuracy: 0.8765 - val_loss: 0

#**Output of the Solution**

In [0]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [0]:
from PIL import Image
from skimage import transform
test_dir = os.path.join(dataset_dir, 'test')

image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
  #print(image_name)
  image_dir = os.path.join(test_dir,image_name)
  img = Image.open(image_dir).convert('RGB')
  img_array = np.array(img)
  img_array = np.array(img_array).astype('float32')/255.
  img_array = transform.resize(img_array, (256, 256, 3))
  img_array = np.expand_dims(img_array, 0)
  pred = model.predict(img_array)
  prediction = np.argmax(model.predict(img_array))   # predicted class
  results[image_name] = prediction

create_csv(results)

In [41]:
!ls

Classification_Dataset	    sample_data
Classification_Dataset.zip  transfer_learning_experiments
results_Nov27_22-16-31.csv


In [0]:
files.download('results_Nov27_22-16-31.csv')